In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import seq2seq

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(seq2seq)  

net = seq2seq.LSTM(char2i, i2char)
net.to(device)
performance = net.fit(source_1, target_1, save_path = "checkpoints/seq2seq_lstm.pt")
net.save_architecture("architectures/seq2seq_lstm.architecture")

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 32
Out embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

Epoch | Train                 | Training time
      | Loss     | Error Rate |
---------------------------------------------
    1 |   2.0073 |     51.419 |         61.72
    2 |   1.3876 |     39.386 |        123.66
    3 |   1.2156 |     35.023 |        185.37
    4 |   1.1222 |     32.770 |        247.20
    5 |   1.0612 |     31.309 |        309.04


In [9]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(source_1[:5], max_predictions = 30)

print(log_probabilities)
net.tensor2text(indexes)

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 32
Out embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

tensor([-29.8569, -25.1782, -18.4017, -20.7909, -30.5338], device='cuda:0')


["<START>g-tm is the King's Game (Pawn ",
 '<START>t Gutenberg-tm to mater (Beent',
 "<START> Gutenberg-tm and the King's G",
 "<START>Gutenberg-tm and the King's Gu",
 '<START>enterg-tm terem is a pawn and ']

In [10]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(source_1[:5])

print(log_probabilities)
net.tensor2text(indexes)

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 32
Out embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

tensor([-38.6331, -51.1987, -42.4235, -36.5803, -48.5462], device='cuda:0')


["<START>m's Lagks. Lege can\n",
 '<START>M Gajlesser, I. 106;',
 '<START> is onoty, altyerb,\n',
 '<START>by\n\nThis 2\n\n     Hur',
 '<START>entmmASbet, An 19E\n\n']

In [11]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(source_1[:5])

print(log_probabilities)
[net.tensor2text(t) for t in indexes]

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 32
Out embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

tensor([[-10.5431, -10.8892, -11.7336, -12.2599, -12.3056],
        [-10.2664, -11.0697, -11.0847, -11.7097, -11.7684],
        [-10.8425, -11.2511, -11.9066, -12.5845, -12.6030],
        [-10.7781, -11.0780, -11.4138, -11.5825, -11.6716],
        [-12.5781, -12.6168, -15.6725, -15.8160, -16.0275]], device='cuda:0')


[['<START>ct Gutenberg-tm and ',
  '<START>ct Gutenberg-tm the ',
  '<START>ct Gutenberg-tm whic',
  '<START>ct Gutenberg-tm is t',
  '<START>ct Gutenberg-tm is n'],
 ['<START>t Gutenberg-tm which',
  '<START>t Gutenberg-tm there',
  '<START>t Gutenberg-tm white',
  '<START>t Gutenberg-tm and P',
  '<START>t Gutenberg-tm the K'],
 ['<START> Gutenberg-tm which ',
  '<START> Gutenberg-tm where ',
  '<START> Gutenberg-tm and th',
  '<START> Gutenberg-tm wheres',
  '<START> Gutenberg-tm and Bl'],
 ['<START>Gutenberg.\n\n        ',
  '<START>Gutenberg.\n\n       B',
  '<START>Gutenberg.\n\n      Wh',
  '<START>Gutenberg.\n\n      Bl',
  '<START>Gutenberg.\n\n     Whi'],
 ['<START>utenberg-tm bect Gut',
  '<START>utenberg-tm betenber',
  '<START>utenberg-tm bect Got',
  '<START>utenberg-tm bectinge',
  '<START>utenberg-tm betion a']]

In [12]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)

idx, log_probabilities = net.predict(source_1[:5])

net.tensor2text(idx)

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 32
Out embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487



['<START>ct Gutenbe',
 '<START>t Gutenber',
 '<START> Gutenberg',
 '<START>Gutenberg-',
 '<START>utenberg-t']